In [24]:
import os
import google.generativeai as genai
import math
import requests
import csv
import pandas as pd
import re
import os
import configparser
import json
import unicodedata
import re
import time
import numpy as np

In [25]:
df = pd.read_csv("E_T3kE.csv")
df.columns = ['id', 'eid', 'content', 'vector', 'page_number', 'split_id', 'word_count', 'head1', 'head_type', 'relation', 'tail', 'tail_type']
df

In [26]:
org = pd.read_csv('org.csv', sep=";", on_bad_lines="skip", engine="python")
org = org.drop('Unnamed: 3', axis=1)
org = org.drop(columns=['eid'])
org = org.drop_duplicates()
org

In [27]:
triples = pd.merge(df, org, left_on=['head1'], right_on=['ori'], how='left')
triples = triples.drop_duplicates()
triples['head1'] = np.where(triples['head1'] == triples['ori'], triples['cam'], triples['head1'])
triples = triples.drop(columns=['ori', 'cam'])
triples = triples.dropna()
triples = triples.drop_duplicates()
triples

In [28]:
triples = pd.merge(triples, org, left_on=['tail'], right_on=['ori'], how='left')
triples = triples.drop_duplicates()
triples['tail'] = np.where(triples['tail'] == triples['ori'], triples['cam'], triples['tail'])
triples = triples.drop_duplicates()
triples = triples.drop(columns=['ori', 'cam'])
triples

In [29]:
#triples.to_csv("triplesLLM.csv", index=False, encoding="utf-8")

DBpedia

In [48]:
import requests

def annotate(text, confidence=0.5, support=20):
    # URL del endpoint de DBpedia Spotlight
    endpoint = "https://api.dbpedia-spotlight.org/en/annotate"
    headers = {"Accept": "application/json"}
    params = {
        "text": text,
        "confidence": confidence,
        "support": support,
    }

    # Realizar la solicitud a la API
    try:
        response = requests.get(endpoint, headers=headers, params=params)
        if response.status_code != 200:
            return -1
        data = response.json()        
        # Extraer la entidad con más relación
        return data["Resources"][0]["@URI"]
            
    except Exception as e:
        return 0


        
# Ejemplo de uso
# text = "ecuador y quito su capital"
# entities = annotate(text)

In [49]:
cValidos = 0 
cFallidos = 0
cTotal   = 0
entities = []


df = triples[['head1']]
df['head1'] = df['head1'].str.lower()
df = df.drop_duplicates(subset='head1', keep='first')


In [50]:

for x in range(0,5290):
    cTotal += 1
    print(x+1, end=" ")
    time.sleep(3) 

    try:
        i = df.iloc[x]
        print(i.head1, end=" ")
        enti = annotate(i.head1)
        if enti != 0:
            entities.append([i.head1, enti])
            cValidos += 1
            print("S")
        elif enti == -1:  
            break
        else:
            cFallidos += 1
            print("N")
    except Exception as e:
        # Imprimir el error o manejarlo de alguna otra forma
        cFallidos += 1
        print("N")

In [51]:
print("Total de entidades con respuesta : ",cValidos)
print("Total de entidades sin respuesta : ",cFallidos)
print("Total de entidades               : ",cTotal)


In [55]:
#T = pd.DataFrame(entities)
#T.to_csv("1dbpedia.csv", index=False)
T = pd.read_csv("dbpedia.csv")
T.columns = ['org', 'cam']
T

In [56]:
tf = triples
tf['head1'] = tf['head1'].str.lower()
tf['tail'] = tf['tail'].str.lower()
T['org'] = T['org'].str.lower()
tf = tf.merge(T, how='left', left_on='head1', right_on='org')
tf['head1'] = tf['cam'].where(tf['cam'].notna(), tf['head1'])
tf = tf.drop(columns=['org', 'cam'])
tf = tf.merge(T, how='left', left_on='tail', right_on='org')
tf['tail'] = tf['cam'].where(tf['cam'].notna(), tf['tail'])
tf = tf.drop(columns=['org', 'cam'])
tf.to_csv("triplesLLMcompletas.csv", index=False)


In [57]:
tf